In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install python_speech_features

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from __future__ import print_function
import sys
import os
import numpy as np


In [ ]:
from python_speech_features import mfcc
from sklearn import preprocessing
import scipy.io.wavfile as wav




In [ ]:
def tf_x_init(fan_in, fan_out, dtype=np.float32):
    k =  np.sqrt(6.0 / (fan_in + fan_out))
    return tf.random_uniform((fan_in, fan_out), minval=-k, maxval=k, dtype=dtype)


def sample_bernoulli(probs):
    return tf.nn.relu(tf.sign(probs - tf.random_uniform(tf.shape(probs))))



In [ ]:

class RBM:
    def __init__(self,
                 n_visible,
                 n_hidden,
                 learning_rate=0.01,
                 momentum=0.95,
                 err_function='mse',
                 use_tqdm=False,
                 # DEPRECATED:
                 tqdm=None):
        if not 0.0 <= momentum <= 1.0:
            raise ValueError('momentum should be in range [0, 1]')

       

        self._use_tqdm = use_tqdm
        self._tqdm = None

        if use_tqdm or tqdm is not None:
            from tqdm import tqdm
            self._tqdm = tqdm

        self.n_visible = n_visible
        self.n_hidden = n_hidden
        self.learning_rate = learning_rate
        self.momentum = momentum

        self.x = tf.placeholder(tf.float32, [None, self.n_visible])
        self.y = tf.placeholder(tf.float32, [None, self.n_hidden])

        self.w = tf.Variable(tf_x_init(self.n_visible, self.n_hidden), dtype=tf.float32)
        self.visible_bias = tf.Variable(tf.zeros([self.n_visible]), dtype=tf.float32)
        self.hidden_bias = tf.Variable(tf.zeros([self.n_hidden]), dtype=tf.float32)

        self.delta_w = tf.Variable(tf.zeros([self.n_visible, self.n_hidden]), dtype=tf.float32)
        self.delta_visible_bias = tf.Variable(tf.zeros([self.n_visible]), dtype=tf.float32)
        self.delta_hidden_bias = tf.Variable(tf.zeros([self.n_hidden]), dtype=tf.float32)

        self.update_weights = None
        self.update_deltas = None
        self.compute_hidden = None
        self.compute_visible = None
        self.compute_visible_from_hidden = None

        self._initialize_vars()

        assert self.update_weights is not None
        assert self.update_deltas is not None
        assert self.compute_hidden is not None
        assert self.compute_visible is not None
        assert self.compute_visible_from_hidden is not None

        
        self.compute_err = tf.reduce_mean(tf.square(self.x - self.compute_visible))

        init = tf.global_variables_initializer()
        self.sess = tf.Session()
        self.sess.run(init)

    def _initialize_vars(self):
        pass

    def get_err(self, batch_x):
        return self.sess.run(self.compute_err, feed_dict={self.x: batch_x})



    def partial_fit(self, batch_x):
        self.sess.run(self.update_weights + self.update_deltas, feed_dict={self.x: batch_x})

    def fit(self,
            data_x,
            n_epoches=10,
            batch_size=10,
            shuffle=True,
            verbose=True):
        assert n_epoches > 0

        n_data = data_x.shape[0]

        if batch_size > 0:
            n_batches = n_data // batch_size + (0 if n_data % batch_size == 0 else 1)
        else:
            n_batches = 1

        if shuffle:
            data_x_cpy = data_x.copy()
            inds = np.arange(n_data)
        else:
            data_x_cpy = data_x

        errs = []

        for e in range(n_epoches):
            if verbose and not self._use_tqdm:
                print('Epoch: {:d}'.format(e))

            epoch_errs = np.zeros((n_batches,))
            epoch_errs_ptr = 0

            if shuffle:
                np.random.shuffle(inds)
                data_x_cpy = data_x_cpy[inds]

            r_batches = range(n_batches)

            if verbose and self._use_tqdm:
                r_batches = self._tqdm(r_batches, desc='Epoch: {:d}'.format(e), ascii=True, file=sys.stdout)

            for b in r_batches:
                batch_x = data_x_cpy[b * batch_size:(b + 1) * batch_size]
                self.partial_fit(batch_x)
                batch_err = self.get_err(batch_x)
                epoch_errs[epoch_errs_ptr] = batch_err
                epoch_errs_ptr += 1

            if verbose:
                err_mean = epoch_errs.mean()
                if self._use_tqdm:
                    self._tqdm.write('Train error: {:.4f}'.format(err_mean))
                    self._tqdm.write('')
                else:
                    print('Train error: {:.4f}'.format(err_mean))
                    print('')
                sys.stdout.flush()

            errs = np.hstack([errs, epoch_errs])

        return errs

    def get_weights(self):
        return self.sess.run(self.w),\
            self.sess.run(self.visible_bias),\
            self.sess.run(self.hidden_bias)

    def save_weights(self, filename, name):
        saver = tf.train.Saver({name + '_w': self.w,
                                name + '_v': self.visible_bias,
                                name + '_h': self.hidden_bias})
        return saver.save(self.sess, filename)

    def set_weights(self, w, visible_bias, hidden_bias):
        self.sess.run(self.w.assign(w))
        self.sess.run(self.visible_bias.assign(visible_bias))
        self.sess.run(self.hidden_bias.assign(hidden_bias))

    def load_weights(self, filename, name):
        saver = tf.train.Saver({name + '_w': self.w,
                                name + '_v': self.visible_bias,
                                name + '_h': self.hidden_bias})
        saver.restore(self.sess, filename)


In [ ]:

class GBRBM(RBM):
    def __init__(self, n_visible, n_hidden, **kwargs):
       

        RBM.__init__(self, n_visible, n_hidden, **kwargs)

    def _initialize_vars(self):
        hidden_p = tf.nn.sigmoid(tf.matmul(self.x, self.w) + self.hidden_bias)
        visible_recon_p = tf.matmul(sample_bernoulli(hidden_p), tf.transpose(self.w)) + self.visible_bias

        

        hidden_recon_p = tf.nn.sigmoid(tf.matmul(visible_recon_p, self.w) + self.hidden_bias)

        positive_grad = tf.matmul(tf.transpose(self.x), hidden_p)
        negative_grad = tf.matmul(tf.transpose(visible_recon_p), hidden_recon_p)

        def f(x_old, x_new):
            return self.momentum * x_old +\
                   self.learning_rate * x_new * (1 - self.momentum) / tf.to_float(tf.shape(x_new)[0])

        delta_w_new = f(self.delta_w, positive_grad - negative_grad)
        delta_visible_bias_new = f(self.delta_visible_bias, tf.reduce_mean(self.x - visible_recon_p, 0))
        delta_hidden_bias_new = f(self.delta_hidden_bias, tf.reduce_mean(hidden_p - hidden_recon_p, 0))

        update_delta_w = self.delta_w.assign(delta_w_new)
        update_delta_visible_bias = self.delta_visible_bias.assign(delta_visible_bias_new)
        update_delta_hidden_bias = self.delta_hidden_bias.assign(delta_hidden_bias_new)

        update_w = self.w.assign(self.w + delta_w_new)
        update_visible_bias = self.visible_bias.assign(self.visible_bias + delta_visible_bias_new)
        update_hidden_bias = self.hidden_bias.assign(self.hidden_bias + delta_hidden_bias_new)

        self.update_deltas = [update_delta_w, update_delta_visible_bias, update_delta_hidden_bias]
        self.update_weights = [update_w, update_visible_bias, update_hidden_bias]

        self.compute_hidden = tf.nn.sigmoid(tf.matmul(self.x, self.w) + self.hidden_bias)
        self.compute_visible = tf.matmul(self.compute_hidden, tf.transpose(self.w)) + self.visible_bias
        self.compute_visible_from_hidden = tf.matmul(self.y, tf.transpose(self.w)) + self.visible_bias


In [ ]:
# data preprocessor
# Here Bulk data is preprocessed to use in training


In [ ]:
shift = 1
merge_limit=4
dataset = []


#name of the file where urbm model learned weights and biases will be saved.
save_weights_path = "/content/gdrive/My Drive/SpeakerVerification/SavedModels/urbm_model" 


lr  = 0.001
input_dim =80
hidden_dim = 400
batch_size = 100
nb_epoch = 100

# this is the directory where all test wav data
# each file corresponds to each speaker
path_dir = "/content/gdrive/My Drive/SpeakerVerification/Data/" 
file_list=os.listdir(path_dir)

print(file_list)

['file_example_WAV_2MG.wav', 'taunt.wav', 'StarWars60.wav', 'StarWars3.wav', 'ortalking.wav', '3peopletalking.wav', 'OSR_us_000_0010_8k.wav', 'OSR_us_000_0011_8k.wav', 'OSR_us_000_0012_8k.wav', 'OSR_us_000_0013_8k.wav', 'OSR_us_000_0030_8k.wav', 'OSR_us_000_0031_8k.wav', 'OSR_us_000_0032_8k.wav', 'OSR_us_000_0034_8k.wav', 'OSR_us_000_0035_8k.wav', 'OSR_us_000_0036_8k.wav']


In [ ]:
# create small segements for tracking audio file ... Target speaker are already extracted..
tracking_file = '3peopletalking.wav'
path=os.path.join(path_dir,tracking_file)

(rate,sig) = wav.read(path)

mfcc_feat = mfcc(sig,rate,numcep=20,nfft=1200)

total_points = mfcc_feat.shape[0]  # total_points 
small_segment_length = 3000        # length of a small segment in ms
mfcc_point_length = 25              # in ms
shift_length = 10 

# number of mfcc features per small segement 
mfcc_points_per_segment =  ( small_segment_length - mfcc_point_length ) // shift_length + 1   

print("total points: ", end="")
print(total_points)

print("small segment length: ", end="")
print(small_segment_length)

print("mfcc points per segment: ", end="")
print(mfcc_points_per_segment)

segment_list = []
for i in range(0, total_points, mfcc_points_per_segment):
    if(i+mfcc_points_per_segment>=total_points):    # the last segment can be less than normal size
        break
    else:
        segment = np.array(mfcc_feat[ i : i + mfcc_points_per_segment - 1 ])

    # print(segment)
    segment_list.append(segment)

small_segments = np.array(segment_list)

print(small_segments.shape)
# print(small_segments)


total points: 10911
small segment length: 3000
mfcc points per segment: 298
(36, 297, 20)


In [ ]:
# utility function to calculate divergence shape
def D(s1,s2):
    C1=np.cov(s1)
    C2=np.cov(s2)

    C1_inv = np.linalg.inv(C1)
    C2_inv = np.linalg.inv(C2)

    sub = np.subtract(C1 , C2)
    inv_sub=np.subtract(C1_inv ,C2_inv)

    res = np.multiply(sub,inv_sub)

    trace = 0.5 * np.trace(res)

    return trace




In [ ]:
# detecting speaker change points and large segments
alpha = 2 
S=small_segments

cum_div = prev_d = D( S[0] , S[1] )

start = 0 
large_segments = []
for i in range(S.shape[0]-2):
    thres = alpha * cum_div / i ;
    cur_d = D(S[i],S[i+1])
    next_d = D(S[i],S[i+2])

    if(cur_d>next_d and cur_d>prev_d and cur_d > thres):  # speaker change occurs
        ls = S[start]
        for j in range(start+1,i):
            ls = np.concatenate((ls,S[j]),axis=0)
        
        large_segments.append(ls)
        start=i

        prev_d= D(S[i],S[i+1])
        i+=1

        cum_div = 0
    else:
        prev_d = cur_d
    
    cum_div +=cur_d;

print(len(large_segments))
print(large_segments[0].shape)
    



In [ ]:
 merge_limit = 4 
 refined_segments=[]  
 for i in range(len(large_segments)):
    mfcc_feat=large_segments[i] 
    length  = mfcc_feat.shape[0] 
    # print(length)
    dataset = []
    for i in range(length-merge_limit+1):
        temp=list(mfcc_feat[i])+list(mfcc_feat[i+1])+list(mfcc_feat[i+2])+list(mfcc_feat[i+3])
        dataset.append(temp)
    # print(len(temp))

    # per speaker augmented data
    augmented_dataset=np.array(dataset)                          # dataset
    augmented_dataset_mvn=preprocessing.scale(augmented_dataset) # augmented dataset
    # print(augmented_dataset_mvn.shape)
    data=augmented_dataset_mvn

    refined_segments.append(data)

# print(len(refined_segments))
# print(refined_segments[0].shape)

In [ ]:
# training for each large segments. or RBM adaptation for each large segments
for i in range(len(refined_segments)):

    data = refined_segments[i]

    gbrbm = GBRBM(n_visible=input_dim, n_hidden=hidden_dim, learning_rate=lr, momentum=0.95, use_tqdm=True)
    gbrbm.load_weights(save_weights_path,'weights')
    errs = gbrbm.fit(data, n_epoches=nb_epoch, batch_size=batch_size)
    name = 'weights'+f ;
    #save each speaker into different folder
    gbrbm.save_weights(save_weights_path+'track'+str(i),'weights'+'track'+str(i))

In [ ]:
# RBM supervector generation ....
import numpy as np

supervector_list=[]

for i in range(len(refined_segments)):
    

    gbrbm = GBRBM(n_visible=input_dim, n_hidden=hidden_dim, learning_rate=lr, momentum=0.95, use_tqdm=True)
    gbrbm.load_weights(save_weights_path+'track'+str(i),'weights'+'track'+str(i))
    w,b_bias,h_bias = gbrbm.get_weights()

    w=np.reshape(w,80*400)
    supervector=np.concatenate((w,b_bias,h_bias),axis=None)
    # print(supervector.size)
    # supervector=preprocessing.scale(supervector) # mean normalization of supervectors
    # print(supervector)
    supervector_list.append(supervector)

# single speaker 
gbrbm.load_weights(save_weights_path+'ortalking.wav','weights'+'ortalking.wav')
w,b_bias,h_bias = gbrbm.get_weights()

w=np.reshape(w,80*400)
supervector=np.concatenate((w,b_bias,h_bias),axis=None)
# print(supervector.size)
# supervector=preprocessing.scale(supervector) # mean normalization of supervectors
# print(supervector)
supervector_list.append(supervector)
# # fit current size of the ram
# trunk_sv_list=[]
# for item in supervector_list:
#     new_item=item[:15000]
#     trunk_sv_list.append(new_item)
# trunk_vectors= np.array(trunk_sv_list)
# print(trunk_vectors.shape)
# #

supervector_data=np.array(supervector_list)
#normalization
supervector_data=preprocessing.scale(supervector_data, axis=0)

# gbrbm = GBRBM(n_visible=input_dim, n_hidden=hidden_dim, learning_rate=lr, momentum=0.95, use_tqdm=True)
# gbrbm.load_weights(save_weights_path,'weights')
# w,b_bias,h_bias = gbrbm.get_weights()


# w=np.reshape(w,80*400)
# urbm_supervector=np.concatenate((w,b_bias,h_bias),axis=None)
# print(supervector.size)
# u_vector=preprocessing.scale(urbm_supervector) # mean normalization of supervectors

INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/SpeakerVerification/SavedModels/urbm_modeltrack0
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/SpeakerVerification/SavedModels/urbm_modeltrack1
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/SpeakerVerification/SavedModels/urbm_modeltrack2
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/SpeakerVerification/SavedModels/urbm_modeltrack3
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/SpeakerVerification/SavedModels/urbm_modeltrack4
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/SpeakerVerification/SavedModels/urbm_modeltrack5
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/SpeakerVerification/SavedModels/urbm_modeltrack6
INFO:tensorflow:Restoring parameters from /content/gdrive/My Drive/SpeakerVerification/SavedModels/urbm_modelortalking.wav


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_data.py:173: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_data.py:190: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


In [ ]:
# PCA whitening and dimensionality reduction  ...and finally RBM vectors for segments
import numpy as np
from sklearn.decomposition import PCA
pca = PCA()
X=supervector_data
pca.fit(X)
rbm_vectors= pca.transform(X)

print(rbm_vectors.shape)
print(rbm_vectors)
print(len(rbm_vectors))

(8, 8)
[[-1.75298214e+01  1.30813522e+01  3.55296745e+01 -5.45067406e+01
   1.08028488e+02 -6.16300507e+01 -4.28915138e+01  1.37751922e-05]
 [-7.78304291e+01 -1.18709541e+02 -4.47407067e-01  2.63786926e+01
  -1.33960905e+01 -6.53126297e+01  7.38083420e+01  1.31222885e-04]
 [-1.03989738e+02 -7.69521942e+01 -4.91601219e+01  6.13261642e+01
  -9.37566185e+00  5.15351372e+01 -7.32220383e+01 -5.42867929e-06]
 [ 2.89654083e+01 -4.63039474e+01  8.64199829e+01 -1.08294006e+02
  -7.66407318e+01  2.51792088e+01 -1.77386818e+01  7.79678812e-05]
 [-3.03564529e+01  4.74591522e+01 -2.39005184e+01 -3.52842827e+01
   5.44207497e+01  9.90444107e+01  6.24765434e+01  6.79846853e-05]
 [-3.35199432e+01  1.36038040e+02 -9.98653183e+01 -2.95356598e+01
  -5.82829666e+01 -4.82962875e+01 -4.29363251e+00  7.63870776e-05]
 [ 1.87369289e+01  9.88492966e+01  1.12727715e+02  1.09787422e+02
  -1.43250341e+01  1.73302710e-01  3.84322429e+00 -6.82468526e-05]
 [ 2.15524002e+02 -5.34621506e+01 -6.13040276e+01  3.01284065e

In [ ]:
from scipy import spatial
for i in range(len(rbm_vectors)-1):
    print(1-spatial.distance.cosine(rbm_vectors[i],rbm_vectors[7]))


-0.20680926740169525
-0.24124018847942352
-0.32762065529823303
-0.014096343889832497
-0.24394436180591583
-0.22100403904914856
-0.11463432759046555
